In [1]:
import pandas as pd
import requests
import json

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [2]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsidGhpbmtfaWQiOiIxNDEyMzIzMSIsImN1c3RvbWVyX3Byb2R1Y3RfaG9sZGluZyI6WyJaWl9OTF9SRUciLCJaWl9TQ19EREFGTCIsIlpaX1NDX0RERlJOT0VOVCJdLCJzaXRlIjoiSGVyYWxkU3VuIiwiaGFzX2NvbnNlbnRlZF90b190YyI6dHJ1ZSwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMC0wNy0xMVQwNzo1Mjo0Mi40NjBaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE1OTQ0NTM5NjIsImV4cCI6MTYwMTc5Nzk2MiwiYXRfaGFzaCI6InRESkRxZUJnRWpBa2N5dFNKUjdCZkEiLCJub25jZSI6IkFmTy5YSn5ZfklOaHh5M2dUZ0hyTC50QUdoRmt6ZWJjIn0.P01TNTizPXHobT3Yt8Md3gTUNkK7I9opZMilZ_iAHsU6-YHwuRQo0jjIdyqtsTvxhBzGpCsEeEez7h2C1kOnrsIF57m3O6DRnRoEeNyB52vP--z7A-zx1KrY2EuawqAVeWc5-mQCcYohSWmx75yhi0bQD3PoPbP6jNxUnmYT3O11_AbnZAM5UkB6jaAcYpsfUVXl6cB_z_R1iuWJvt5fSHJg53LPtL4waV61YEaVCbl8h9ON1Cz13-vsd_2dndxwC_CqA9gdLEw2HUF195ebtDMvVN6raYojvRZGg1EI9HVcCK0sP4q6VUspGDMZm2L97UaKCw7SYwA5zQ6WJ46bJw",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token", data=payload)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"token_type":"Bearer","expires_in":604800,"access_token":"0ba7d23b9621f2e272a19f87a39fd27c816b8fee","refresh_token":"41270ff34cb8dd8c58e00d8393447fce32d1d95b"}'

## PLAYER LIST

In [3]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/players-cf", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']]
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team
0,151,293957,Brodie,Grundy,COL
0,469,290528,Max,Gawn,MEL
0,755,295467,Jack,Macrae,WBD
0,76,293535,Lachie,Neale,BRL
0,244,291570,Nat,Fyfe,FRE
...,...,...,...,...,...
0,818,1016214,Ayce,Taylor,ADE
0,821,1016097,Henry,Crauford,ESS
0,822,1020224,Tom,Hird,ESS
0,825,297668,Nicholas,Reid,WCE


## LADDER AND FIXTURE DATA

In [4]:
# specify round
round_number = 9

s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/ladderAndFixtures?round={}&scores=true".format(round_number), headers=headers)
match_json = json.loads(s.text)

## COACH IDs AND NAMES

In [5]:
df_coaches = pd.DataFrame()

for coach in match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])

df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,86078,9806,Lester,HoweIMetYourMother
0,45168,5524,Luke,The Dragon's Laird
0,180650,33301,Jordan,"What's Up, Doch?"
0,21562,186,Mark,GetRichOrDieTryin
0,300791,182,Paul,Coronaviney
0,71052,9867,James,Bad Boys For Fyfe
0,8258,9856,Simon,It's always Sonny
0,10857,189,Anthony,The Full Bonty


## PLAYER MATCH RESULTS

In [6]:
def get_player_score_data(on_field_status):
    df_player_results = pd.DataFrame()
    if on_field_status == True:
        scoring_tag = 'scoring'
    else:
        scoring_tag = 'nonscoring'
    df_player_line = pd.DataFrame()
    for player in coach['userTeam']['scores'][scoring_tag]:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        df_player_line = pd.DataFrame(
            {
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results
    

df_player_match_results = pd.DataFrame()

for coach in match_json['ladder']:
    coach_id = coach['userTeam']['user']['id'],
    coach_team_id = coach['userTeam']['id'],
    coach_first_name = coach['userTeam']['user']['first_name'],
    coach_team_name = coach['userTeam']['teamname'],
    df_player_line = pd.DataFrame()
    df_on_field_players = get_player_score_data(True)
    df_off_field_players = get_player_score_data(False)
    df_all_players = pd.concat([df_on_field_players, df_off_field_players])
    df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results

,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,points,on_field,feed_id,first_name,last_name,team
0,86078,9806,Lester,HoweIMetYourMother,620,184,True,290847,Dustin,Martin,RIC
1,86078,9806,Lester,HoweIMetYourMother,94,149,True,990704,Patrick,Cripps,CAR
2,86078,9806,Lester,HoweIMetYourMother,366,145,True,291800,Tom,Stewart,GEE
3,86078,9806,Lester,HoweIMetYourMother,807,135,True,296296,Dom,Sheed,WCE
4,86078,9806,Lester,HoweIMetYourMother,341,121,True,290550,Cameron,Guthrie,GEE
...,...,...,...,...,...,...,...,...,...,...,...
171,10857,189,Anthony,The Full Bonty,576,45,True,997142,Peter,Ladhams,PTA
172,10857,189,Anthony,The Full Bonty,322,73,False,298524,Lachie,Weller,GCS
173,10857,189,Anthony,The Full Bonty,286,87,False,293713,Brandon,Ellis,GCS
174,10857,189,Anthony,The Full Bonty,413,0,False,280109,Callan,Ward,GWS


In [7]:
df_player_match_results.to_csv('outputs/sc_rounds/round_{}_player_match_results.csv'.format(round_number))

## FIXTURE RESULTS

In [8]:
df_fixture_results = pd.DataFrame()
for result in match_json['fixtures']:
    df_line = pd.DataFrame(
        {
            'round_number': [result['round']],
            'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
            'home_team_id': [result['user_team1']['id']],
            'home_team_name': [result['user_team1']['teamname']],
            'home_team_coach': [result['user_team1']['user']['first_name']],
            'home_team_points': [result['user_team1']['stats'][0]['points']],
            'away_team_id': [result['user_team2']['id']],
            'away_team_name': [result['user_team2']['teamname']],
            'away_team_coach': [result['user_team2']['user']['first_name']],
            'away_team_points': [result['user_team2']['stats'][0]['points']],
        }
    )
    df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,9,Paul vs Anthony,182,Coronaviney,Paul,1637,189,The Full Bonty,Anthony,1630
0,9,Lester vs Mark,9806,HoweIMetYourMother,Lester,1875,186,GetRichOrDieTryin,Mark,1667
0,9,James vs Luke,9867,Bad Boys For Fyfe,James,1497,5524,The Dragon's Laird,Luke,1743
0,9,Jordan vs Simon,33301,"What's Up, Doch?",Jordan,1619,9856,It's always Sonny,Simon,1617


In [9]:
df_fixture_results.to_csv('outputs/sc_rounds/round_{}_fixture_results.csv'.format(round_number))

## CURRENTLY LISTED TEAMS

In [10]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,coach_id,coach_team_id,coach_first_name,coach_team_name
0,151,owner,186.0,NaN,293957,Brodie,Grundy,COL,21562,186,Mark,GetRichOrDieTryin
1,469,owner,33301.0,NaN,290528,Max,Gawn,MEL,180650,33301,Jordan,"What's Up, Doch?"
2,755,owner,33301.0,NaN,295467,Jack,Macrae,WBD,180650,33301,Jordan,"What's Up, Doch?"
3,76,owner,5524.0,NaN,293535,Lachie,Neale,BRL,45168,5524,Luke,The Dragon's Laird
4,244,owner,9867.0,NaN,291570,Nat,Fyfe,FRE,71052,9867,James,Bad Boys For Fyfe
...,...,...,...,...,...,...,...,...,...,...,...,...
171,49,owner,9806.0,NaN,1002312,Zac,Bailey,BRL,86078,9806,Lester,HoweIMetYourMother
172,645,owner,33301.0,NaN,992374,Dan,Butler,STK,180650,33301,Jordan,"What's Up, Doch?"
173,379,owner,33301.0,NaN,294508,Aidan,Corr,GWS,180650,33301,Jordan,"What's Up, Doch?"
174,580,owner,186.0,NaN,293603,Sam,Mayes,PTA,21562,186,Mark,GetRichOrDieTryin


## CURRENT FREE AGENTS + WAIVER

In [11]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team
32,513,free_agent,NaN,NaN,280921,Ben,Cunnington,NTH
43,5,free_agent,NaN,NaN,294307,Brad,Crouch,ADE
45,676,free_agent,NaN,NaN,293846,Sebastian,Ross,STK
47,108,free_agent,NaN,NaN,261299,Matthew,Kreuzer,CAR
53,116,free_agent,NaN,NaN,297907,Nic,Newman,CAR
...,...,...,...,...,...,...,...,...
821,818,free_agent,NaN,NaN,1016214,Ayce,Taylor,ADE
822,821,free_agent,NaN,NaN,1016097,Henry,Crauford,ESS
823,822,free_agent,NaN,NaN,1020224,Tom,Hird,ESS
824,825,free_agent,NaN,NaN,297668,Nicholas,Reid,WCE


In [12]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')